![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [56]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [57]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like
cols = list(ds_jobs_transformed.columns)
binary_cols = [col for col in cols if ds_jobs_transformed[col].value_counts().shape[0] == 2]

# requirement 1
ds_jobs_transformed[binary_cols] = ds_jobs_transformed[binary_cols].astype('bool')    
print(ds_jobs_transformed.dtypes)

student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                   bool
dtype: object


In [58]:
# requirement 2
int_cols = list(ds_jobs_transformed.select_dtypes('int').columns)
int_cols
ds_jobs_transformed[int_cols] = ds_jobs_transformed[int_cols].astype('int32')
ds_jobs_transformed.dtypes

student_id                  int32
city                       object
city_development_index    float64
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                   bool
dtype: object

In [59]:
# requirement 3
float_cols = list(ds_jobs_transformed.select_dtypes('float64').columns)
float_cols
ds_jobs_transformed[float_cols] = ds_jobs_transformed[float_cols].astype('float16')
ds_jobs_transformed.dtypes

student_id                  int32
city                       object
city_development_index    float16
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                   bool
dtype: object

In [60]:
# requirement 4
object_cols = list(ds_jobs_transformed.select_dtypes('object').columns)
object_cols
ds_objects = ds_jobs_transformed[object_cols]
nominal_cols = ['city', 'gender', 'company_type', 'major_discipline']
# ds_jobs_transformed[nominal_cols] = ds_jobs_transformed[nominal_cols].astype('category')
# ds_jobs_transformed.dtypes
nominal_columns = [(col, ds_objects[col].nunique()) for col in ds_objects.columns]
nominal_columns
ds_jobs_transformed[nominal_cols] = ds_jobs_transformed[nominal_cols].astype('category')




In [61]:
# requirement 5
ordinal_cols = list(set(object_cols) - set(nominal_cols))
ordinal_cols
ds_ordinals = ds_objects[ordinal_cols]
last_job_categories = ['never', '1', '2', '3', '4', '>4']
# experience_cols = list(ds_ordinals['experience'].dropna().unique())
experience_col = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
# company_size = list(ds_ordinals['company_size'].dropna().unique())
company_size = ['<10', '10-49', '50-99', '100-499', '500-999','1000-4999','5000-9999', '10000+']
# education_level = list(ds_ordinals['education_level'].dropna().unique())
education_level = ['Primary School', 'High School', 'Graduate', 'Masters', 'PhD']
enrolled_university = ['no_enrollment', 'Part time course', 'Full time course']
dict_values = [(col, ds_ordinals[col].value_counts()) for col in ds_ordinals.columns]


# answer
ds_jobs_transformed['last_new_job'] = pd.Categorical(ds_jobs_transformed['last_new_job'], categories=last_job_categories, ordered=True)
ds_jobs_transformed['experience'] = pd.Categorical(ds_jobs_transformed['experience'], categories=experience_col, ordered=True)
ds_jobs_transformed['company_size'] = pd.Categorical(ds_jobs_transformed['company_size'], categories=company_size, ordered=True)
ds_jobs_transformed['education_level'] = pd.Categorical(ds_jobs_transformed['education_level'], categories=education_level, ordered=True)
ds_jobs_transformed['enrolled_university'] = pd.Categorical(ds_jobs_transformed['enrolled_university'], categories=enrolled_university, ordered=True)

ds_jobs_transformed.dtypes

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                    bool
dtype: object

In [62]:
# req 6
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['company_size'] >= '1000-4999') & (ds_jobs_transformed['experience'] >= '10')]
ds_jobs_transformed.sample(10)


,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
12260,15903,city_103,0.919922,Female,True,no_enrollment,Masters,STEM,11,10000+,Pvt Ltd,1,61,False
10220,11772,city_102,0.804199,Male,True,no_enrollment,Masters,STEM,11,1000-4999,Pvt Ltd,1,15,True
6595,28527,city_103,0.919922,Male,True,no_enrollment,Graduate,STEM,13,10000+,Pvt Ltd,1,46,False
5563,9931,city_103,0.919922,NaN,True,no_enrollment,Graduate,Humanities,>20,1000-4999,Public Sector,>4,234,False
11109,20582,city_136,0.896973,Male,True,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,1,26,False
5202,20316,city_149,0.688965,Male,True,Full time course,Masters,STEM,10,5000-9999,Pvt Ltd,1,9,False
883,16796,city_103,0.919922,Male,True,no_enrollment,Graduate,STEM,16,1000-4999,Pvt Ltd,1,54,False
550,26291,city_75,0.938965,Male,True,no_enrollment,Graduate,STEM,11,1000-4999,Pvt Ltd,>4,72,False
2983,15514,city_71,0.883789,Male,True,no_enrollment,Masters,STEM,15,1000-4999,Pvt Ltd,1,6,False
12737,30539,city_103,0.919922,NaN,True,no_enrollment,Masters,STEM,>20,1000-4999,Public Sector,1,22,False


In [63]:
print(ds_jobs_transformed.info(), ds_jobs.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2185 non-null   category
 6   education_level         2069 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9)